# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'06-05-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'06-05-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-06-06 04:24:03,33.93911,67.709953,77963,3104,58265.0,16594.0,Afghanistan,200.273112,3.981376
1,NaN,NaN,NaN,Albania,2021-06-06 04:24:03,41.15330,20.168300,132374,2451,129627.0,296.0,Albania,4599.833206,1.851572
2,NaN,NaN,NaN,Algeria,2021-06-06 04:24:03,28.03390,1.659600,130681,3510,90995.0,36176.0,Algeria,298.011156,2.685930
3,NaN,NaN,NaN,Andorra,2021-06-06 04:24:03,42.50630,1.521800,13758,127,13527.0,104.0,Andorra,17806.251213,0.923099
4,NaN,NaN,NaN,Angola,2021-06-06 04:24:03,-11.20270,17.873900,35594,794,28866.0,5934.0,Angola,108.299488,2.230713


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/27/21,5/28/21,5/29/21,5/30/21,5/31/21,6/1/21,6/2/21,6/3/21,6/4/21,6/5/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,68366,69130,70111,70761,71838,72977,74026,75119,76628,77963
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,132264,132285,132297,132309,132315,132337,132351,132360,132372,132374
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,127926,128198,128456,128725,128913,129218,129640,129976,130361,130681


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/27/21,5/28/21,5/29/21,5/30/21,5/31/21,6/1/21,6/2/21,6/3/21,6/4/21,6/5/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,2869,2881,2899,2919,2944,2973,3007,3034,3068,3104
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,2447,2448,2449,2450,2451,2451,2451,2451,2451,2451
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,3448,3455,3460,3465,3472,3480,3490,3497,3504,3510


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/27/21,5/28/21,5/29/21,5/30/21,5/31/21,6/1/21,6/2/21,6/3/21,6/4/21,6/5/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,56962,57119,57281,57450,57629,57741,57963,58070,58144,58265
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,129042,129097,129215,129308,129431,129473,129521,129566,129598,129627
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,89040,89232,89419,89625,89839,90057,90281,90517,90767,90995


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,5/27/21,5/28/21,5/29/21,5/30/21,5/31/21,6/1/21,6/2/21,6/3/21,6/4/21,6/5/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,7141,7142,7142,7142,7142,7150,7161,7172,7186,7186
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,21606,21620,21620,21620,21620,21661,21674,21684,21698,21698
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,2333,2334,2334,2334,2334,2337,2340,2343,2342,2342


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,5/27/21,5/28/21,5/29/21,5/30/21,5/31/21,6/1/21,6/2/21,6/3/21,6/4/21,6/5/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,110,110,110,110,110,111,111,111,112,112
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,310,311,311,311,311,311,311,312,312,312
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,58,59,59,59,59,59,59,59,59,59


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
653,1001.0,Autauga,Alabama,US,2021-06-06 04:24:03,32.539527,-86.644082,7186,112,NaN,NaN,"Autauga, Alabama, US",12862.231291,1.558586
690,1075.0,Lamar,Alabama,US,2021-06-06 04:24:03,33.779950,-88.096680,1458,37,NaN,NaN,"Lamar, Alabama, US",10561.390800,2.537723
691,1077.0,Lauderdale,Alabama,US,2021-06-06 04:24:03,34.901719,-87.656247,9537,245,NaN,NaN,"Lauderdale, Alabama, US",10284.808420,2.568942


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,33357080,597377,0.0
India,28694879,344082,26795549.0
Brazil,16907425,472531,14944069.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,33357080,597377,0.0,32759703.0,2021-06-06 04:24:03,37.950547,-91.419547
India,28694879,344082,26795549.0,1555248.0,2021-06-06 04:24:03,23.088275,81.806127
Brazil,16907425,472531,14944069.0,1490825.0,2021-06-06 04:24:03,-12.669522,-48.480493
France,5769291,110135,395384.0,5263772.0,2021-06-06 04:24:03,6.430808,-34.730285
Turkey,5282594,48068,5154771.0,79755.0,2021-06-06 04:24:03,38.963700,35.243300


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,3794802,63441,0.0
Texas,2962395,51701,0.0
Florida,2332867,36973,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,3794802,63441,0.0,3731361.0,2021-06-06 04:24:03,37.843962,-120.728594
Texas,2962395,51701,0.0,2910694.0,2021-06-06 04:24:03,31.660643,-98.653069
Florida,2332867,36973,0.0,2295894.0,2021-06-06 04:24:03,28.940755,-82.700744
New York,2105375,53429,0.0,2051946.0,2021-06-06 04:24:03,42.544151,-75.474183
Illinois,1384823,25314,0.0,1359509.0,2021-06-06 04:24:03,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,1244917,24400,0.0
Illinois,Cook,554813,10357,0.0
Arizona,Maricopa,551998,10111,0.0
Florida,Miami-Dade,502548,6472,0.0
Texas,Harris,401469,6474,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,1244917,24400,0.0,1220517.0,2021-06-06 04:24:03,34.308284,-118.228241,6037.0
Illinois,Cook,554813,10357,0.0,544456.0,2021-06-06 04:24:03,41.841448,-87.816588,17031.0
Arizona,Maricopa,551998,10111,0.0,541887.0,2021-06-06 04:24:03,33.348359,-112.491815,4013.0
Florida,Miami-Dade,502548,6472,0.0,496076.0,2021-06-06 04:24:03,25.611236,-80.551706,12086.0
Texas,Harris,401469,6474,0.0,394995.0,2021-06-06 04:24:03,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-03,75119,132360,129976,13752,35140,1262,3884447,222978,30141,645834,...,4515778,304411,100997,4,238013,8063,309333,6767,97388,39092
2021-06-04,76628,132372,130361,13758,35307,1263,3915397,223050,30157,646167,...,4521922,308490,101198,4,239252,8364,309644,6773,98376,39144
2021-06-05,77963,132374,130681,13758,35594,1263,3939024,223143,30173,646438,...,4527577,312703,101436,4,240714,8580,309853,6780,99540,39168


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-03,3034,2451,3497,127,784,42,79873,4448,910,10627,...,128075,4460,693,1,2689,49,3509,1325,1288,1604
2021-06-04,3068,2451,3504,127,788,42,80411,4452,910,10631,...,128086,4516,695,1,2698,51,3511,1325,1297,1605
2021-06-05,3104,2451,3510,127,794,42,80867,4454,910,10636,...,128099,4583,696,1,2708,53,3516,1325,1303,1605


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-03,58070,129566,90517,13527,28646,1213,3438437,213813,23601,630731,...,15517,263733,96889,3,218956,3085,301443,3484,92320,36680
2021-06-04,58144,129598,90767,13527,28802,1214,3465137,214059,23601,631138,...,15524,267776,97077,3,219971,3242,301544,3511,92514,36690
2021-06-05,58265,129627,90995,13527,28866,1214,3497500,214268,23603,630458,...,15528,271481,97517,3,221108,3310,301670,3550,93036,36698


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,5/27/21,5/28/21,5/29/21,5/30/21,5/31/21,6/1/21,6/2/21,6/3/21,6/4/21,6/5/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,7141,7142,7142,7142,7142,7150,7161,7172,7186,7186
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,1449,1450,1450,1450,1450,1452,1454,1457,1458,1458
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,9510,9511,9511,9511,9511,9520,9524,9526,9537,9537
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,3076,3076,3076,3076,3076,3081,3084,3086,3091,3091
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,16104,16115,16115,16115,16115,16135,16153,16171,16190,16190


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-06-03,545028,70408,0,882691,341889,3792962,545002,347748,108957,49,...,863620,2959171,406825,24240,3512,676041,438544,162111,674939,60549
2021-06-04,545585,70469,0,883037,342142,3794129,545769,347891,109005,49,...,863840,2961257,407113,24252,3512,676300,439263,162232,675150,60628
2021-06-05,545585,70469,0,883314,342345,3794802,546500,347891,109070,49,...,863840,2962395,407419,24252,3512,676586,440057,162232,675150,60628


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2021-06-03        7172   21684    2343  2665   6894    1236   2223   14655   
2021-06-04        7186   21698    2342  2666   6906    1237   2223   14668   
2021-06-05        7186   21698    2342  2666   6906    1237   2223   14668   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-06-03         3692     1866  ...    2850    663     3227      797   
2021-06-04         3696     1866  ...    2856    667     3230      797   
2021-06-05         3696     1866  ...    2856    667     3230      797   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-06-03           4558  3785  2287          0      924    655  
2021-06-04           4576  3787  2289          0      924    655  
2021-06-05           4576  3787  2289          0      924    655  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-06-03,11188,369,0,17653,5842,63345,6603,8245,1668,0,...,12472,51633,2308,255,28,11216,5821,2807,7923,720
2021-06-04,11198,369,0,17673,5845,63401,6611,8246,1674,0,...,12476,51676,2311,256,28,11222,5836,2813,7954,720
2021-06-05,11198,369,0,17698,5846,63441,6613,8246,1677,0,...,12476,51701,2311,256,28,11229,5836,2813,7954,720


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-06-03         111     312      59   64    139      42     71     325   
2021-06-04         112     312      59   64    139      42     71     325   
2021-06-05         112     312      59   64    139      42     71     325   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-06-03          124       45  ...      30     11       31        7   
2021-06-04          124       45  ...      30     11       31        7   
2021-06-05          124       45  ...      30     11       31        7   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-06-03             39     9    13          1       26      5  
2021-06-04             39     9    13          1       26      5  
2021-06-05             39     9    13          1       26      5  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-03,0.014765,0.000068,0.002592,0.000582,0.005149,0.000000,0.008383,0.000485,0.000133,0.000437,...,0.001149,0.009575,0.002690,0.0,0.005314,0.024524,0.000961,0.001184,0.008544,0.001563
2021-06-04,0.020088,0.000091,0.002962,0.000436,0.004752,0.000792,0.007968,0.000323,0.000531,0.000516,...,0.001361,0.013400,0.001990,0.0,0.005206,0.037331,0.001005,0.000887,0.010145,0.001330
2021-06-05,0.017422,0.000015,0.002455,0.000000,0.008129,0.000000,0.006034,0.000417,0.000531,0.000419,...,0.001251,0.013657,0.002352,0.0,0.006111,0.025825,0.000675,0.001034,0.011832,0.000613


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-03,0.008979,0.0,0.002006,0.0,0.005128,0.0,0.006972,0.000450,0.0,0.000565,...,0.000141,0.015020,0.002894,0.0,0.005610,0.000000,0.000570,0.001512,0.003115,0.003127
2021-06-04,0.011206,0.0,0.002002,0.0,0.005102,0.0,0.006736,0.000899,0.0,0.000376,...,0.000086,0.012556,0.002886,0.0,0.003347,0.040816,0.000570,0.000000,0.006988,0.000623
2021-06-05,0.011734,0.0,0.001712,0.0,0.007614,0.0,0.005671,0.000449,0.0,0.000470,...,0.000101,0.014836,0.001439,0.0,0.003706,0.039216,0.001424,0.000000,0.004626,0.000000


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-03,0.001846,0.000347,0.002614,0.001481,0.013515,0.000000,0.008542,0.001100,0.000254,0.000725,...,0.000193,0.014518,0.003314,0.0,0.005317,0.000000,0.000764,0.003456,0.002302,0.000518
2021-06-04,0.001274,0.000247,0.002762,0.000000,0.005446,0.000824,0.007765,0.001151,0.000000,0.000645,...,0.000451,0.015330,0.001940,0.0,0.004636,0.050891,0.000335,0.007750,0.002101,0.000273
2021-06-05,0.002081,0.000224,0.002512,0.000000,0.002222,0.000000,0.009340,0.000976,0.000085,-0.001077,...,0.000258,0.013836,0.004532,0.0,0.005169,0.020975,0.000418,0.011108,0.005642,0.000218


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-06-03,0.000790,0.000753,NaN,0.000365,0.000577,0.000283,0.001553,0.000201,0.000422,0.0,...,0.000338,0.000878,0.000844,0.000330,0.0,0.000382,0.001981,0.000889,0.000296,0.001687
2021-06-04,0.001022,0.000866,NaN,0.000392,0.000740,0.000308,0.001407,0.000411,0.000441,0.0,...,0.000255,0.000705,0.000708,0.000495,0.0,0.000383,0.001640,0.000746,0.000313,0.001305
2021-06-05,0.000000,0.000000,NaN,0.000314,0.000593,0.000177,0.001339,0.000000,0.000596,0.0,...,0.000000,0.000384,0.000752,0.000000,0.0,0.000423,0.001808,0.000000,0.000000,0.000000


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-06-03      0.001536  0.000461  0.001282  0.000000  0.000726  0.000000   
2021-06-04      0.001952  0.000646 -0.000427  0.000375  0.001741  0.000809   
2021-06-05      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-06-03      0.001351  0.000956  0.001356  0.000536  ...  0.002110   
2021-06-04      0.000000  0.000887  0.001083  0.000000  ...  0.002105   
2021-06-05      0.000000  0.000000  0.000000  0.000000  ...  0.000000   

Province_State                                                              \
Admin2            Platte  Sheridan Sublette Sweetwater     Teton     Uinta   
2021-06-03      0.012214  0.002174      0.0   0.002419  0.000000  0.003510   
2021-06-04      0.006033  0.000930      0.0   0.003949  0.000528  0.000875   
2021-06-05      0.000000  0.000000      0.0   0.000000  0.000000  0.000000   

Province_State                              
Admin2         Unassigned  Washakie Weston  
2021-06-03            NaN -0.001081    0.0  
2021-06-04            NaN  0.000000    0.0  
2021-06-05            NaN  0.000000    0.0  

[3 rows x 3336 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-06-03,0.001881,0.0,NaN,0.000283,0.001200,0.000806,0.001973,-0.000243,0.001200,NaN,...,0.000562,0.001125,0.001302,0.000000,0.0,0.000892,0.003448,0.002500,0.003674,0.0
2021-06-04,0.000894,0.0,NaN,0.001133,0.000514,0.000884,0.001212,0.000121,0.003597,NaN,...,0.000321,0.000833,0.001300,0.003922,0.0,0.000535,0.002577,0.002138,0.003913,0.0
2021-06-05,0.000000,0.0,NaN,0.001415,0.000171,0.000631,0.000303,0.000000,0.001792,NaN,...,0.000000,0.000484,0.000000,0.000000,0.0,0.000624,0.000000,0.000000,0.000000,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State   Alabama                                               \
Admin2           Autauga   Baldwin Barbour Bibb Blount Bullock Butler   
2021-06-03      0.000000  0.003215     0.0  0.0    0.0     0.0    0.0   
2021-06-04      0.009009  0.000000     0.0  0.0    0.0     0.0    0.0   
2021-06-05      0.000000  0.000000     0.0  0.0    0.0     0.0    0.0   

Province_State                              ... Wyoming                  \
Admin2           Calhoun Chambers Cherokee  ...    Park Platte Sheridan   
2021-06-03      0.003086      0.0      0.0  ...     0.0    0.0      0.0   
2021-06-04      0.000000      0.0      0.0  ...     0.0    0.0      0.0   
2021-06-05      0.000000      0.0      0.0  ...     0.0    0.0      0.0   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-06-03          0.0        0.0   0.0   0.0        0.0      0.0    0.0  
2021-06-04          0.0        0.0   0.0   0.0        0.0      0.0    0.0  
2021-06-05          0.0        0.0   0.0   0.0        0.0      0.0    0.0  

[3 rows x 3336 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-03,0.014584,0.000091,0.002630,0.000755,0.005563,0.000453,0.008588,0.000438,0.000248,0.000504,...,0.000988,0.010728,0.002418,1.894781e-14,0.005396,0.028946,0.001003,0.001192,0.007292,0.001176
2021-06-04,0.017336,0.000091,0.002796,0.000596,0.005158,0.000623,0.008278,0.000381,0.000389,0.000510,...,0.001174,0.012064,0.002204,9.473903e-15,0.005301,0.033138,0.001004,0.001039,0.008718,0.001253
2021-06-05,0.017379,0.000053,0.002625,0.000298,0.006643,0.000311,0.007156,0.000399,0.000460,0.000465,...,0.001212,0.012860,0.002278,4.736952e-15,0.005706,0.029482,0.000840,0.001036,0.010275,0.000933


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-03,0.009505,0.000049,0.002236,1.925050e-12,0.006845,1.070174e-06,0.007226,0.000633,1.221367e-19,0.000641,...,0.000098,0.014059,0.001961,0.0,0.005534,0.008194,0.000881,0.001164,0.002264,0.001986
2021-06-04,0.010356,0.000025,0.002119,9.625249e-13,0.005973,5.350872e-07,0.006981,0.000766,6.106837e-20,0.000509,...,0.000092,0.013307,0.002423,0.0,0.004440,0.024505,0.000725,0.000582,0.004626,0.001305
2021-06-05,0.011045,0.000012,0.001916,4.812624e-13,0.006794,2.675436e-07,0.006326,0.000608,3.053419e-20,0.000490,...,0.000097,0.014072,0.001931,0.0,0.004073,0.031860,0.001075,0.000291,0.004626,0.000652


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-03,0.002511,0.000412,0.002519,0.001683,0.008840,0.000406,0.008700,0.000991,0.000263,0.000881,...,0.000302,0.013608,0.003167,4.547474e-13,0.005154,0.006458,0.000767,0.003559,0.001685,0.000645
2021-06-04,0.001893,0.000329,0.002640,0.000842,0.007143,0.000615,0.008232,0.001071,0.000131,0.000763,...,0.000377,0.014469,0.002554,2.273737e-13,0.004895,0.028675,0.000551,0.005654,0.001893,0.000459
2021-06-05,0.001987,0.000277,0.002576,0.000421,0.004683,0.000308,0.008786,0.001023,0.000108,-0.000157,...,0.000317,0.014153,0.003543,1.136868e-13,0.005032,0.024825,0.000484,0.008381,0.003768,0.000338


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-06-03,0.000804,0.000713,NaN,0.000535,0.000519,0.000271,0.001239,0.000271,0.000347,7.493834e-135,...,0.000357,0.000698,0.000633,0.000324,0.004346,0.000311,0.001797,0.001079,0.000301,0.001648
2021-06-04,0.000913,0.000790,NaN,0.000464,0.000629,0.000289,0.001323,0.000341,0.000394,3.746917e-135,...,0.000306,0.000702,0.000670,0.000410,0.002173,0.000347,0.001718,0.000913,0.000307,0.001476
2021-06-05,0.000456,0.000395,NaN,0.000389,0.000611,0.000233,0.001331,0.000171,0.000495,1.873458e-135,...,0.000153,0.000543,0.000711,0.000205,0.001086,0.000385,0.001763,0.000456,0.000153,0.000738


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-06-03      0.001302  0.000628  0.001131  0.000049  0.000865  0.000608   
2021-06-04      0.001627  0.000637  0.000352  0.000212  0.001303  0.000709   
2021-06-05      0.000813  0.000318  0.000176  0.000106  0.000651  0.000354   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-06-03      0.000754  0.000678  0.001992  0.000546  ...  0.002531   
2021-06-04      0.000377  0.000783  0.001538  0.000273  ...  0.002318   
2021-06-05      0.000188  0.000391  0.000769  0.000137  ...  0.001159   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-06-03      0.006958  0.001762  0.000331   0.002792  0.000040  0.002872   
2021-06-04      0.006496  0.001346  0.000166   0.003371  0.000284  0.001873   
2021-06-05      0.003248  0.000673  0.000083   0.001685  0.000142  0.000937   

Province_State                                     
Admin2         Unassigned      Washakie    Weston  
2021-06-03           -1.0  3.372775e-06  0.000206  
2021-06-04           -1.0  1.686388e-06  0.000103  
2021-06-05           -1.0  8.431938e-07  0.000052  

[3 rows x 3336 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-06-03,0.001308,2.452021e-07,NaN,0.000455,0.000690,0.000586,0.001484,0.000072,0.001364,NaN,...,0.000629,0.000885,0.001045,1.215871e-07,0.009259,0.000928,0.002796,0.001755,0.002993,0.000182
2021-06-04,0.001101,1.226011e-07,NaN,0.000794,0.000602,0.000735,0.001348,0.000097,0.002480,NaN,...,0.000475,0.000859,0.001173,1.960845e-03,0.004630,0.000732,0.002686,0.001946,0.003453,0.000091
2021-06-05,0.000550,6.130053e-08,NaN,0.001104,0.000387,0.000683,0.000825,0.000048,0.002136,NaN,...,0.000238,0.000671,0.000586,9.804226e-04,0.002315,0.000678,0.001343,0.000973,0.001727,0.000046


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                  \
Admin2           Autauga   Baldwin   Barbour          Bibb        Blount   
2021-06-03      0.001138  0.001633  0.000136  5.925093e-11  6.995527e-09   
2021-06-04      0.005073  0.000816  0.000068  2.962547e-11  3.497764e-09   
2021-06-05      0.002537  0.000408  0.000034  1.481273e-11  1.748882e-09   

Province_State                                                            ...  \
Admin2               Bullock    Butler   Calhoun  Chambers      Cherokee  ...   
2021-06-03      2.326180e-08  0.000058  0.001571  0.002033  6.465934e-16  ...   
2021-06-04      1.163090e-08  0.000029  0.000786  0.001016  3.232967e-16  ...   
2021-06-05      5.815451e-09  0.000014  0.000393  0.000508  1.616484e-16  ...   

Province_State       Wyoming                                            \
Admin2                  Park        Platte      Sheridan      Sublette   
2021-06-03      3.651485e-24  1.482124e-40  2.757830e-26  1.211539e-46   
2021-06-04      1.825742e-24  7.410619e-41  1.378915e-26  6.057697e-47   
2021-06-05      9.128712e-25  3.705309e-41  6.894574e-27  3.028848e-47   

Province_State                                                       \
Admin2            Sweetwater         Teton         Uinta Unassigned   
2021-06-03      4.015597e-07  3.867461e-34  3.880511e-11  -0.000020   
2021-06-04      2.007798e-07  1.933731e-34  1.940255e-11  -0.000009   
2021-06-05      1.003899e-07  9.668654e-35  9.701277e-12  -0.000004   

Province_State                              
Admin2              Washakie        Weston  
2021-06-03      7.527498e-39  1.598672e-37  
2021-06-04      3.763749e-39  7.993362e-38  
2021-06-05      1.881875e-39  3.996681e-38  

[3 rows x 3336 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN
